In [3]:
def add(a, b):
    return a+b

def grad_add(grad, a, b):
    return grad, grad

def mul(a, b):
    return a*b

def grad_mul(grad, a, b):
    return grad*b, grad*a

input1, input2, input3, input4 = (1., 2., 6., 4.)

'''inference'''
dummy1 = add(input1, input2)
dummy2 = add(input3, input4)
dummy3 = mul(dummy1, dummy2)

'''caculate gradient'''
grad3_dummy1, grad3_dummy2 = grad_mul(1, dummy1, dummy2)
grad2_a, grad2_b = grad_add(grad3_dummy1, input1, input2)
grad1_a, grad1_b = grad_add(grad3_dummy2, input3, input4)

'''output'''
print dummy3
print grad3_dummy1, grad3_dummy2
print grad2_a, grad2_b
print grad1_a, grad1_b



30.0
10.0 3.0
10.0 10.0
3.0 3.0


In [1]:
import tensorflow as tf

input1 = tf.placeholder(shape=None, dtype=tf.float32, 
                        name='input1')
input2 = tf.placeholder(shape=None, dtype=tf.float32, 
                        name='input2')
input3 = tf.placeholder(shape=None, dtype=tf.float32, 
                        name='input3')
input4 = tf.placeholder(shape=None, dtype=tf.float32, 
                        name='input4')

'''inference'''
dummy3 = (input1+input2)*(input3+input4)

'''caculate gradient'''
input1_grad, input2_grad, input3_grad, input4_grad =(
    tf.gradients(dummy3, input1),
    tf.gradients(dummy3, input2),
    tf.gradients(dummy3, input3),
    tf.gradients(dummy3, input4)
)

tf.summary.scalar('accuracy', 1.0)
merged = tf.summary.merge_all()

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('./stuff',graph=sess.graph)
    sess.run(merged)
    (in_1_grad, in_2_grad, in_3_grad, in_4_grad
    ) = sess.run([input1_grad, input2_grad, 
                  input3_grad, input4_grad], 
                 feed_dict={input1:1., input2:2., 
                            input3:6., input4:4.})
    print in_1_grad, in_2_grad
    print in_3_grad, in_4_grad

[10.0] [10.0]
[3.0] [3.0]


In [1]:
import tensorflow as tf
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])  # v.name == "foo/v:0"
    w = tf.get_variable("w", [1])  # w.name == "foo/w:0"
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v")  # The same as v above.

print w
print v
print v1

Tensor("foo/w/read:0", shape=(1,), dtype=float32)
Tensor("foo/v/read:0", shape=(1,), dtype=float32)
Tensor("foo/v/read:0", shape=(1,), dtype=float32)


In [2]:
import tensorflow as tf
with tf.variable_scope("foo") as scope:
    v = tf.get_variable("v", [1])  # v.name == "foo/v:0"
    w = tf.get_variable("w", [1])  # w.name == "foo/w:0"
    scope.reuse_variables()
    v1 = tf.get_variable("v")  # The same as v above.

print w
print v
print v1

Tensor("foo/w/read:0", shape=(1,), dtype=float32)
Tensor("foo/v/read:0", shape=(1,), dtype=float32)
Tensor("foo/v/read:0", shape=(1,), dtype=float32)


In [1]:
import tensorflow as tf

with tf.variable_scope('foo'):
    with tf.variable_scope('dummy') as sp:
        v = tf.get_variable('v', shape=[1, 2])
    v2 = tf.get_variable('v2', shape=[1, 1])

print v.name
print v2.name


foo/dummy/v:0
foo/v2:0


In [1]:
import tensorflow as tf

with tf.variable_scope('foo'):
    with tf.variable_scope('dummy') as sp:
        v = tf.get_variable('v', initializer=[1, 2])
        sp.reuse_variables()
        v2 = tf.get_variable('v', initializer=[4, 5], dtype=tf.int32)
    v3 = tf.get_variable('v3', [1, 1])
    
foo_var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='foo/v3')
dummy_var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='foo/dummy')

print v.name
print v2.name
print v3.name

print [o.name for o in foo_var_list]
print [o.name for o in dummy_var_list]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run([v, v2])

foo/dummy/v:0
foo/dummy/v:0
foo/v3:0
[u'foo/v3:0']
[u'foo/dummy/v:0']
[array([1, 2], dtype=int32), array([1, 2], dtype=int32)]


In [1]:
import tensorflow as tf

with tf.variable_scope('foo'):
    with tf.variable_scope('dummy') as sp:
        v = tf.get_variable('v', initializer=[[2., 3.]])
    
dummy_var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='foo/dummy/v')
dummy_var = dummy_var_list[0]
dummy_add = dummy_var+4.25

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run([v, dummy_add])

[array([[ 2.,  3.]], dtype=float32), array([[ 6.25,  7.25]], dtype=float32)]


In [5]:
c1 = tf.constant(1.)
c2 = tf.constant(2.)
print c1, c2
print [o.name for o in tf.global_variables()]
print [o.name for o in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='foo/v3')]

Tensor("Const_7:0", shape=(), dtype=float32) Tensor("Const_8:0", shape=(), dtype=float32)
[u'foo/dummy/v:0', u'foo/v3:0']
[u'foo/v3:0']


In [10]:
import numpy as np
aa = np.random.rand(1)

for i in range(5):
    print aa
    
    

[ 0.12619646]
[ 0.12619646]
[ 0.12619646]
[ 0.12619646]
[ 0.12619646]


In [7]:
import tensorflow as tf
choice = tf.placeholder(shape=[], dtype=tf.bool)
out = tf.cond(choice, lambda: tf.zeros([2, 2]), 
              lambda: tf.ones([3, 3]))
with tf.Session() as sess:
    print sess.run(out, feed_dict={choice: True})
    print 
    print sess.run(out, feed_dict={choice: False})

[[ 0.  0.]
 [ 0.  0.]]

[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
